<a href="https://colab.research.google.com/github/samarth-web/Big_Data_Project/blob/main/big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q pyspark pandas matplotlib


In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"samarthj204","key":"543cc1df9b5aa7879c6f8d0117abab12"}'}

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



In [6]:
!kaggle datasets download maharshipandya/-spotify-tracks-dataset


Dataset URL: https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset
License(s): ODbL-1.0
  0% 0.00/8.17M [00:00<?, ?B/s]
100% 8.17M/8.17M [00:00<00:00, 553MB/s]


In [7]:
!unzip /content/-spotify-tracks-dataset.zip -d spotify_data


Archive:  /content/-spotify-tracks-dataset.zip
  inflating: spotify_data/dataset.csv  


In [8]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder.appName("SpotifySparkHive").config("spark.sql.warehouse.dir", "./spark_warehouse").enableHiveSupport().getOrCreate())
spark

In [9]:
spark.sql("CREATE DATABASE IF NOT EXISTS spotify_music")
spark.sql("USE spotify_music")
spark.sql("SHOW DATABASES")

DataFrame[namespace: string]

In [10]:
path = "spotify_data/dataset.csv"

sp_read = spark.read.option("header", True).option("inferSchema", True).csv(path)


In [11]:
sp_read.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- track_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- track_genre: string (nullable = true)



In [12]:
sp_read.write.mode("overwrite").saveAsTable("spotify_tracks")

In [13]:
spark.sql("SHOW TABLES").show()

+-------------+--------------+-----------+
|    namespace|     tableName|isTemporary|
+-------------+--------------+-----------+
|spotify_music|spotify_tracks|      false|
+-------------+--------------+-----------+



In [14]:
spark.sql("SELECT * FROM spotify_tracks LIMIT 5")

DataFrame[_c0: int, track_id: string, artists: string, album_name: string, track_name: string, popularity: string, duration_ms: string, explicit: string, danceability: string, energy: string, key: string, loudness: string, mode: string, speechiness: string, acousticness: string, instrumentalness: double, liveness: string, valence: string, tempo: double, time_signature: double, track_genre: string]

**Exploratory Data Analysis**

1.) Average Dancibility of all Songs

In [15]:
spark.sql("SELECT AVG(danceability) as avg_dance FROM spotify_tracks").show()

+------------------+
|         avg_dance|
+------------------+
|31.798216588376423|
+------------------+



2.) Average Dancibility of songs by genre


In [16]:
from pyspark.sql import functions as F
numeric_cols = [
    "popularity", "duration_ms", "danceability", "energy",
    "loudness", "speechiness", "acousticness",
    "liveness", "valence", "tempo"
]
sp_new = sp_read

for col in numeric_cols:
  sp_new = sp_new.withColumn(col,F.col(col).cast("double"))

sp_new.printSchema()
spark.sql("CREATE DATABASE IF NOT EXISTS spotify_music")
spark.sql("USE spotify_music")
sp_new.write.mode("overwrite").saveAsTable("spotify_tracks")


root
 |-- _c0: integer (nullable = true)
 |-- track_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- explicit: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- track_genre: string (nullable = true)



In [17]:
spark.table("spotify_tracks").printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- track_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- explicit: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- track_genre: string (nullable = true)



In [18]:
spark.sql("""
SELECT track_genre, ROUND(AVG(danceability),3) AS avg_dance, COUNT(*) AS n
FROM spotify_tracks
WHERE track_genre NOT RLIKE '^[0-9.\\-]+$'
GROUP BY track_genre
ORDER BY avg_dance DESC
""").show(truncate=False)

+--------------+---------+----+
|track_genre   |avg_dance|n   |
+--------------+---------+----+
|kids          |0.779    |1000|
|chicago-house |0.766    |999 |
|reggaeton     |0.759    |1000|
|latino        |0.757    |1000|
|reggae        |0.745    |1000|
|hip-hop       |0.736    |1000|
|dancehall     |0.734    |1000|
|minimal-techno|0.729    |1000|
|detroit-techno|0.723    |1000|
|latin         |0.722    |1000|
|children      |0.716    |1000|
|deep-house    |0.71     |1000|
|sad           |0.692    |1000|
|dance         |0.688    |1000|
|study         |0.685    |1000|
|techno        |0.684    |1000|
|j-dance       |0.681    |1000|
|funk          |0.678    |1000|
|disco         |0.677    |1000|
|afrobeat      |0.67     |1000|
+--------------+---------+----+
only showing top 20 rows



**Machine Learning - Predicting Genre of the Track**

In [19]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import functions as F
#the below steps are needed as to do ML on spark, there has to be a label column with numerical datatypes and all the features have to be combined into one column named 'features'

In [20]:
indexer = StringIndexer(inputCol="track_genre", outputCol="label", handleInvalid="skip") #converts the track_genre into a numerical label
indexed_df = indexer.fit(sp_new).transform(sp_new)


In [33]:
feature_cols = ["danceability","energy","loudness","speechiness",
                "acousticness","instrumentalness","liveness","valence",
                "tempo","popularity","duration_ms"]

# Drop rows with nulls in feature columns from the original DataFrame before assembling features
indexed_df_cleaned = indexed_df.na.drop(subset=feature_cols)

assembler = VectorAssembler(inputCols=feature_cols,outputCol='features')
sp_ml = assembler.transform(indexed_df_cleaned).select('features','label')

In [34]:
# Split the cleaned and assembled data
train, test = sp_ml.randomSplit([0.7,0.3], seed = 42)

In [37]:
rf = RandomForestClassifier(featuresCol='features', labelCol='label',numTrees=200, seed=42)
# Drop rows with nulls from the training data before fitting the model
train_cleaned = train.na.drop()
model = rf.fit(train_cleaned)
pred = model.transform(test)

In [36]:
acc = MulticlassClassificationEvaluator(metricName='accuracy').evaluate(pred)
f1 = MulticlassClassificationEvaluator(metricName='f1').evaluate(pred)
print(f"Accuracy: {acc}\nF1 Score: {f1}")

Accuracy: 0.18400468727112934
F1 Score: 0.1289320929754292
